<a href="https://colab.research.google.com/github/Alice751/MNIST/blob/master/Emotion_Recogniton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models
from torch.autograd import Variable
import torch.utils.data.distributed
import torchvision.transforms as transforms
import os
from PIL import Image
import pandas as pd

In [ ]:
!nvidia-smi

Wed Sep  8 11:14:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
pth_path = '/content/drive/MyDrive/data/backbone_ir50_ms1m_epoch63.pth'

train_path = '/content/drive/MyDrive/data/train/african/'
test_path = '/content/drive/MyDrive/data/test/african/'
val_path = '/content/drive/MyDrive/data/val/african/tests/'
model_path_base = '/content/drive/MyDrive/data/models/african/'
model_name = 'afr-resnet50-4emo.pth'

# Train

In [ ]:
BATCH_SIZE = 8
EPOCHS = 50
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    modellrnew = modellr * (0.1 ** (epoch // 50))
    print("lr:", modellrnew)
    for param_group in optimizer.param_groups:
        param_group['lr'] = modellrnew

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    sum_loss = 0
    total_num = len(train_loader.dataset)
    print(total_num, len(train_loader))
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).to(device), Variable(target).to(device)
        output = model(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print_loss = loss.data.item()
        sum_loss += print_loss
        if (batch_idx + 1) % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
                       100. * (batch_idx + 1) / len(train_loader), loss.item()))
    ave_loss = sum_loss / len(train_loader)
    print('epoch:{},loss:{}'.format(epoch, ave_loss))

def val(model, device, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  total_num = len(test_loader.dataset)
  print(total_num, len(test_loader))
  with torch.no_grad():
      for data, target in test_loader:
        data, target = Variable(data).to(device), Variable(target).to(device)
        output = model(data)
        loss = criterion(output, target)
        _, pred = torch.max(output.data, 1)
        correct += torch.sum(pred == target)
        print_loss = loss.data.item()
        test_loss += print_loss
        
      correct = correct.data.item()
      acc = correct / total_num
      avgloss = test_loss / len(test_loader)
      print('\nVal set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            avgloss, correct, len(test_loader.dataset), 100 * acc)) 

In [ ]:
 transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
 
])
 
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

dataset_train = datasets.ImageFolder(train_path, transform)
dataset_test = datasets.ImageFolder(test_path, transform_test)
 
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False)
modellr = 0.0001
# pthfile = r'backbone_ir50_ms1m_epoch63.pth'

criterion = nn.CrossEntropyLoss()
model = torchvision.models.resnet50(pretrained=False)
model.load_state_dict(torch.load(pth_path),False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)
model.to(DEVICE)

optimizer = optim.Adam(model.parameters(), lr=modellr)

In [ ]:
for epoch in range(1, EPOCHS + 1):
  adjust_learning_rate(optimizer, epoch)
  train(model, DEVICE, train_loader, optimizer, epoch)
  val(model, DEVICE, test_loader)
  torch.save(model, model_path_base+model_name)

# Test

In [ ]:
import pandas as pd
base_path = '/content/drive/MyDrive/data/all/african/'
label_path = '/content/drive/MyDrive/data/csvs/african/test_african_0.2.csv'
df = pd.read_csv(label_path,header=None)
df.head()

,0,1
0,afr_angry_13.jpg,0
1,afr_neutral_18.jpg,2
2,afr_neutral_5.jpg,2
3,afr_angry_31.jpg,0
4,afr_neutral_39.jpg,2


In [ ]:
test_imgs = df[0].tolist()
labels = df[1].tolist()

In [ ]:
classes = (0,1, 2, 3)
 
transform_test = transforms.Compose([
    #transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
 
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.load(model_path_base+model_name)
model.eval()
model.to(DEVICE)
i=0
n=0
path=val_path
testList=os.listdir(path)

for j,file in enumerate(test_imgs):
  f = base_path+file
  # print(f)
  n=n+1
  img=Image.open(f)
  img=transform_test(img)
  img.unsqueeze_(0)
  img = Variable(img).to(DEVICE)
  out=model(img)
  # Predict
  _, pred = torch.max(out.data, 1)
  # label = file.split('_')[1]
  label = labels[j]
  if(label == classes[pred.data.item()]):
      i=i+1
  # print('Image Name:{}, predict:{}'.format(file,classes[pred.data.item()]))

print('Test_Accuracy: {}/{} ({:.0f}%)\n'.format(i, n, 100 * i/n))

Test_Accuracy: 37/73 (51%)

